In [487]:
# importa las librerias necesarias 
import pandas as pd
import numpy as np
import pyarrow as pa

In [515]:
# lee el archivo parquet 
df_raw = pd.read_parquet("../data/reservas_hoteles.parquet")
df_raw.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
9983,e2014df8-187f-4ee8-bd15-7b0b57aac983,f91707af-c4ef-414b-984e-3bf73ccdce5a,Laura,Grande,laura.grande@example.com,True,,2025-03-01,2025-03-02,103,NaN,,NaN,
1870,8d7ecf9a-0cc9-42bc-88c9-3d41f799d1e1,de30d057-73d2-4731-9590-7274ff829274,Evelia,Piñeiro,evelia.piñeiro@example.com,False,2025-02-01,2025-03-01,2025-03-02,38,259.65,Hotel Maravilla Real,2.0,Madrid
14008,b9cce0c2-19a9-49e8-961f-08d039ee3fb3,25ada371-e5f4-4158-ac3f-e84ba21a2366,Dorotea,Olivares,dorotea.olivares@example.com,False,2025-02-09,2025-03-01,2025-03-02,19,423.39,Hotel Camino del Sol,2.0,Madrid
4061,83e525b7-7996-4490-b476-c2cd569967a4,03efeff2-4993-43d2-8cd5-64317d345123,Vilma,Jáuregui,vilma.jáuregui@example.com,True,,2025-03-01,2025-03-02,135,NaN,,NaN,
296,d078f387-4fca-40d7-b775-36b08e2d5d08,ddab46ca-d583-4304-aa54-1fd36e1fe433,Joaquina,Valbuena,joaquina.valbuena@example.com,False,2025-02-04,2025-03-01,2025-03-02,36,174.14,Palacio del Sol,4.0,Madrid


In [489]:
# realiza una copia del archivo original
df = df_raw.copy()

In [490]:
# saca la forma del dataframe
df.shape

(15098, 14)

In [491]:
# crea una funcion que genera un reporte con informacion de nulos y tipo de dato de cada una de las columnas 
def info_reporte(dataframe):
    df_report = pd.DataFrame()
    df_report["Numero_nulos"] = dataframe.isnull().sum()
    df_report["Porcentaje_nulos"] = round((dataframe.isnull().sum()/dataframe.shape[0]*100), 2)
    df_report["Tipo_dato"] = dataframe.dtypes
    return df_report

In [492]:
# llama a la funcion aplicandola al dataframe 
info_reporte(df)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
id_reserva,0,0.00,object
id_cliente,0,0.00,object
nombre,0,0.00,object
apellido,0,0.00,object
mail,0,0.00,object
competencia,0,0.00,bool
fecha_reserva,0,0.00,object
inicio_estancia,75,0.50,object
final_estancia,75,0.50,object
id_hotel,0,0.00,int64


In [493]:
# saca los valores únicos de la columna estrellas
df["estrellas"].unique()

array([nan,  1.,  5.,  4.,  3.,  2.])

- Tengo 15098 entradas y contamos con 14 columnas 
- En cuanto al tipo de dato de las columnas, ya se identifican determinadas columnas cuyo tipo de dato es incorrecto, concretamente las columnas de tipo datetime que están en tipo object. Es decir, fecha_reserva, inicio_estancia y final_estancia. El resto parecen estar correctamente. En cuanto a la columna de estrellas, se observa que cuenta con 5 valores únicos, es decir, no recoge números de estrellas que no sean enteros tipo 4.3, sino que solo dispone de los valores: 1,2,3,4,5, con lo que en principio debería estar en tipo integer.
- En un principio, en función de como he planteado una primera estructuración de la BBDD, la única columna faltante sería de la id_ciudad que se generaría al crear la tabla Ciudades. 
- Observación primera de nulos: Contamos con nulos en 4 columnas (inicio_estancia, final_estancia, precio_noche, estrellas)

In [494]:
# convierte el tipo de dato de las 3 columnas de fecha en tipo datetime
col_fechas = ["fecha_reserva", "inicio_estancia", "final_estancia"]
for col in col_fechas:
    df[col] = pd.to_datetime(df[col])

In [495]:
# convierte la columna estrellas de tipo float a tipo integer 
df["estrellas"] = df["estrellas"].astype("Int64")

In [496]:
# comprueba que se han cambiado correctamente los tipos de dato de las columnas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   Int64         
 13  ciudad           15098 non-null  object        
dtypes: Int64(1), bool(1), datetime64[ns](3

In [497]:
# saca el numero de duplicados por todas las columnas que hay en el dataframe
df.duplicated().sum()

np.int64(98)

In [498]:
# elimina los duplicados por todas las columnas del dataframe 
df.drop_duplicates(inplace=True)

In [499]:
# comprueba si hay duplicados por la columna id_reserva
df["id_reserva"].duplicated().sum()

np.int64(0)

In [500]:
# saca la forma del dataframe tras haber eliminado duplicados
df.shape

(15000, 14)

En cuanto a los duplicados, contamos con 98 duplicados por todas las columnas. Es decir, reservas que están completamente repetidas. En este caso, las eliminamos del conjunto de datos  nos quedamos con 15000 reservas. Además, una vez eliminadas estas reservas es importante observar si por el campo "id_reserva" hay algún duplicado, y ¿por qué? pues porque id_reserva es el identificador único de cada reserva (cada registro) y no puede estar repetido. Comprobamos que no hay duplicados por este campo y por lo tanto, todas las reservas son únicas y estaría todo correcto. Por el resto de id como el id_cliente o el id_hotel no hay que comprobarlo ya que por estos campos sí pueden haber duplicados. Pueden haber clientes que hayan realizado más de una reserva o hoteles que hayan sido reservados en más de una ocasión.

En cuanto a los valores nulos, de primeras parecía que únicamente habían 4 columnas con valores nulos (inicio_estancia, final_estancia, precio_noche, estrellas), pero por lo que he observado en estos primeros vistazos sobre el conjunto de datos, las columnas fecha_reserva, nombre_hotel y ciudad también tienen valores faltantes, lo que ocurre es que son espacios en blanco y no se están contando como valores nulos porque pandas no lo está identificando así. Además, parece que los nulos tienen que ver con los hoteles de la competencia. Lo primero que voy a hacer es convertir esos espacios en blanco en valores nulos para hacer un recuento correcto de los mismos.

In [501]:
# reemplaza los valores en blanco por valores nulos 
col_blancos = ["fecha_reserva", "nombre_hotel", "ciudad"]
for col in col_blancos:
    df[col] = df[col].replace("", np.nan)

In [502]:
# llama a la funcion reporte aplicandola al dataframe 
info_reporte(df)

,Numero_nulos,Porcentaje_nulos,Tipo_dato
id_reserva,0,0.00,object
id_cliente,0,0.00,object
nombre,0,0.00,object
apellido,0,0.00,object
mail,0,0.00,object
competencia,0,0.00,bool
fecha_reserva,5172,34.48,datetime64[ns]
inicio_estancia,75,0.50,datetime64[ns]
final_estancia,75,0.50,datetime64[ns]
id_hotel,0,0.00,int64


In [503]:
# recuento de hoteles de competencia 
df[df["competencia"] == True].shape[0]

5172

Observamos como ahora ya si nos está sacando el recuento y el porcentaje de nulos de todas las columnas. De primeras ya es curioso que las columnas fecha_reserva, nombre_hotel, estrellas y ciudad tengan el mismo número de nulos, por lo de que parece que tienen relación con los hoteles de competencia. Y, efectivamente, coincide con el número de reservas que provienen de hoteles de la competencia (5172). Por lo tanto, parecen columnas de las cuales se van a extraer los datos a través de web scraping.

DUDA MARTES: para la ciudad hace falta hacer el web scraping? si siempre es Madrid 

DUDA MARTES: y el dato de fecha de reserva como se saca? de donde y tal?

In [504]:
# saca el numero de valores unicos de las columnas inicio_estancia y final_estancia
df[["inicio_estancia", "final_estancia"]].nunique()

inicio_estancia    1
final_estancia     1
dtype: int64

Por otro lado, en cuanto a las columnas de inicio_estancia y final_estancia, del primer vistazo de los datos, también he podido observar que por lo que parece, ambas disponen de un único valor, y además, lo he comprobado, y es así. Es decir, para todas las reservas, la fecha de inicio de la estancia es la misma, concretamente el 1 de marzo de 2025, y para todas las reservas la fecha de fin de estancia también es la misma, concretamente el 3 de marzo de 2025, es decir, es como si todas las estancias únicamente durasen un día y además todas el mismo día. Esto no tiene sentido y aquí está ocurriendo algo (error de recogida en los datos, todavía no se ha comenzado la recogida de datos de estas dos columnas correctamente etc.).

DUDA MARTES: En este caso, se podrían tomar dos decisiones: eliminar las columnas de la base de datos, o si se quieren mantener, estos nulos que además son un porcentaje muy ínfimo (0,5% en cada columna), se podrían rellenar con la misma fecha, pues esta claro que correctamente (al menos por ahora) no se están recogiendo estas dos fechas.

Por último, la última columna que tiene nulos es el precio_noche. Hay nulos en esta columna para 52 reservas de hoteles propios, y el resto de nulos corresponden a todos los hoteles de la competencia, es decir, un dato igualmente a scrapear en el caso de los hoteles de la competencia. En el caso de los hoteles propios, hay que valorar como tratar estos 52 registros nulos.

In [524]:
# saca el numero de reservas correspondientes con hoteles propios y con precio_noche nulo
numero_reservas = df[(df["competencia"] == False) & (df["precio_noche"].isna())].shape[0]
print(numero_reservas)
numero_hoteles_competencia = df[df["precio_noche"].isna()].shape[0] - numero_reservas
print(numero_hoteles_competencia)

52
5172


In [533]:
# saca los hoteles y el numero de registros por cada hotel donde precio_noche es nulo
df_nulos_precio = df[(df["competencia"] == False) & (df["precio_noche"].isna())]
df_nulos_precio["nombre_hotel"].value_counts()

nombre_hotel
Hotel Sol y Luna          5
Hotel Torre Dorada        5
Hotel Camino del Sol      4
Hotel Mirador Real        4
Hotel Las Estrellas       4
Hotel Monte Verde         4
Hotel Puerta del Cielo    3
Hotel Encanto Real        3
Palacio del Sol           3
Gran Hotel Madrid         3
Hotel Brisas del Mar      2
Hotel Los Almendros       2
Hotel Vista Alegre        2
Hotel Rincón Sereno       2
Hotel Costa Azul          2
Hotel Luz de Madrid       1
Hotel Jardines del Rey    1
Hotel Palacio Imperial    1
Hotel Maravilla Real      1
Name: count, dtype: int64

In [ ]:
# filtra por el hotel "Hotel Sol y Luna"
df[df["nombre_hotel"] == "Hotel Sol y Luna"].sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
9586,7ef26298-3b09-41b8-b02e-40560c3218a7,95ee9397-29cb-4529-a5d3-428ea61ca453,Gonzalo,Raya,gonzalo.raya@example.com,False,2025-02-08,2025-03-01,2025-03-02,36,187.65,Hotel Sol y Luna,4,Madrid
11875,c2d02117-5d7e-4939-a84f-2e8387c12706,6917cf8b-c9cf-4242-8e07-bc22cfd70966,Inés,Sala,inés.sala@example.com,False,2025-02-04,2025-03-01,2025-03-02,33,150.65,Hotel Sol y Luna,3,Madrid
1494,cfd15395-3c49-4aff-a2af-6b2d6e538814,0232fdc8-d44f-4387-b596-0800f4e8e4d7,Palmira,Agullo,palmira.agullo@example.com,False,2025-02-02,2025-03-01,2025-03-02,13,195.55,Hotel Sol y Luna,2,Madrid
10262,6d51ad8b-bfc2-4c6c-98e5-c0fcf7373dd8,3b6be0cc-53ae-4138-9466-5b5593a2612f,Haroldo,Marín,haroldo.marín@example.com,False,2025-02-07,2025-03-01,2025-03-02,49,155.23,Hotel Sol y Luna,2,Madrid
7488,df05e77c-0e1e-4701-a53e-369ef556b48d,00194c55-9178-4224-b94f-d750e101af73,Alexandra,Porcel,alexandra.porcel@example.com,False,2025-02-04,2025-03-01,2025-03-02,13,190.69,Hotel Sol y Luna,1,Madrid


Como se puede observar, para por ejemplo el Hotel Sol y Luna el precio_noche es diferente para cada reserva. Lo que se puede valorar es sustituir los pocos nulos que hay por cada hotel por la media o la mediana (en función de los outliers) de la columna precio_noche para cada hotel. 

HACER ANALISIS DE OUTLIERS PARA DECIDIR SI RELLENAR NULOS CON MEDIA O MEDIANA

DUDA MARTES: sacando esta informacion me he dado cuenta de que hay algo raro porque el Hotel Sol y Luna en Madrid que deberia ser el mismo claro, tiene para cada reserva un id_hotel diferente cuando deberia ser el mismo y también estrellas diferentes. ??????